In [1]:
import os
from groq import Groq
import pandas as pd
import numpy as np

In [2]:
apiKey = "fill_it_here"

In [3]:
client = Groq(
    # api_key=os.environ.get("GROQ_API_KEY"),
    api_key= apiKey,
)

In [4]:
dataset = pd.read_csv("keyphrases/done/keyphrases_SpotifyLlama_Done.csv")

In [5]:
dataset

,Unnamed: 0,content,keyphrases_original,cluster,sentiment,count,keyphrases_reduced,Merged keyphrases_reduced,count merged keyphrases_reduced,kw newly generated,id Mkr,Merged keyphrases_reduced 2,count 2,kw newly generated 2,count 2 off,id Mkr 2
0,0,I was wrong with spotify I can't install it no...,"['user comment', 'Spotify App', 'installation ...",49,0,7,"['Spotify app login issue', 'User account acce...","['Spotify app login issue', 'User account acce...",7,NaN,0,"['Spotify app login issue', 'User account acce...",7.0,NaN,7,0
1,688,I have had my account for three years. Now thi...,"['long-time Spotify user', 'account logout iss...",49,0,7,"['Spotify app login issue', 'User account acce...","['Spotify app login issue', 'User account acce...",7,NaN,0,"['Spotify app login issue', 'User account acce...",7.0,NaN,7,0
2,810,(I have Premium) Randomly logged me out on bot...,"['Spotify Premium', 'logged out', 'phone', 'Xb...",49,0,7,"['Spotify app login issue', 'User account acce...","['Spotify app login issue', 'User account acce...",7,NaN,0,"['Spotify app login issue', 'User account acce...",7.0,NaN,7,0
3,823,I'm paying for premium. You guys booted me off...,"['paying for premium', 'booted off premium', '...",49,0,7,"['Spotify app login issue', 'User account acce...","['Spotify app login issue', 'User account acce...",7,NaN,0,"['Spotify app login issue', 'User account acce...",7.0,NaN,7,0
4,1314,Can't log in at all anymore. Can't seem to cha...,"[""Can't log in"", 'Login issue', 'Unable to cha...",49,0,7,"['Spotify app login issue', 'User account acce...","['Spotify app login issue', 'User account acce...",7,NaN,0,"['Spotify app login issue', 'User account acce...",7.0,NaN,7,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2695,2678,There's a bug where the now playing bar at the...,"['bug', 'now playing bar', 'bottom', 'music co...",298,1,2,"['Spotify app issues', 'Last update problems',...","['Spotify app issues', 'Last update problems',...",2,NaN,416,"['Spotify app issues', 'Last update problems',...",2.0,NaN,2,354
2696,2617,"Can't even play the song I want to play, let u...","[""can't play song"", 'freely control songs', 's...",469,0,1,"[""can't play song"", 'freely control songs', 's...","['Spotify app', 'used to harm people', 'can't ...",2,True,415,"['Spotify app', 'user comment', 'adds shows', ...",10.0,True,14,351
2697,2647,"Works good for me, I also found how to link yo...","['Spotify app functionality', 'user experience...",302,2,1,"['Spotify app functionality', 'user experience...","['Spotify app functionality', 'user experience...",2,True,417,"['Spotify app functionality', 'user experience...",2.0,NaN,2,355
2698,2682,"Bloated, unorganized, and a data-hog. Features...","['Bloated', 'unorganized', 'data-hog', 'playli...",224,0,1,"['Bloated', 'unorganized', 'data-hog', 'playli...","['playlists navigation', 'favorite artists nav...",1,True,418,"['Spotify app', 'user comment', 'adds shows', ...",10.0,True,14,351


In [6]:
score_counts_balanced = dataset['sentiment'].value_counts().sort_index()
print(score_counts_balanced)

sentiment
0    900
1    900
2    900
Name: count, dtype: int64


In [7]:
train_data = dataset[['content', 'sentiment', 'Merged keyphrases_reduced 2']]
train_data['row_number'] = train_data.index

/tmp/ipykernel_2803414/1289041381.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['row_number'] = train_data.index


In [8]:
train_data

,content,sentiment,Merged keyphrases_reduced 2,row_number
0,I was wrong with spotify I can't install it no...,0,"['Spotify app login issue', 'User account acce...",0
1,I have had my account for three years. Now thi...,0,"['Spotify app login issue', 'User account acce...",1
2,(I have Premium) Randomly logged me out on bot...,0,"['Spotify app login issue', 'User account acce...",2
3,I'm paying for premium. You guys booted me off...,0,"['Spotify app login issue', 'User account acce...",3
4,Can't log in at all anymore. Can't seem to cha...,0,"['Spotify app login issue', 'User account acce...",4
...,...,...,...,...
2695,There's a bug where the now playing bar at the...,1,"['Spotify app issues', 'Last update problems',...",2695
2696,"Can't even play the song I want to play, let u...",0,"['Spotify app', 'user comment', 'adds shows', ...",2696
2697,"Works good for me, I also found how to link yo...",2,"['Spotify app functionality', 'user experience...",2697
2698,"Bloated, unorganized, and a data-hog. Features...",0,"['Spotify app', 'user comment', 'adds shows', ...",2698


In [ ]:
list = []
i = 0
for index, row in train_data.iterrows():
    text = row['content']
    topics = row['Merged keyphrases_reduced 2']
    string_message = f"""I need you to evaluate and assess the comment along with the set of topics/keyphrases, categorizing their sentiment into one of five levels: 1 - very negative, 2 - negative, 3 - neutral, 4 - positive, 5 - very positive. Here are examples to guide your classification:


                1. Comment: "Terrible. I won't repeat what the other reviews have said, the continuous ads are so intrusive and time-consuming with the free version. Now that they are joining the likes of other silicone valley politics and starting to censor people, I am officially done with Spotify. I was honestly considering premium to avoid the ads, but then the censorship started."
                Topics / Keyphrases: ['Spotify app issues', 'Invasive ads on free version', 'Commercial increase and bias', 'Censorship and control of information', 'Freedom of speech and fair advertising', 'Spotify's subscription and user choice', 'App controversies and linking issues', 'Silicon Valley politics and app controversies', 'Ad overload and diversity in job advertising', 'Cancel culture and free speech']
                Label: 1

                2. Comment: "ads and limited skips are annoying. you could literally download songs from free music apps and listen to it offline, unlimited skips and choose whatever songs you want. i used Spotify for a few weeks now and i think I'll go back using Music instead of this app."
                Topics / Keyphrases: ['Spotify app issues', 'Terrible user experience', 'Advertisements and skips', 'Music streaming limitations', 'Free version limitations', 'Playlist organization and customization', 'Offline listening and playback', 'Volume control and multitasking', 'Premium subscription features and limitations', 'User interface and feature requests']
                Label: 1

                3. Comment: "Poor music site with bad ethics. They were better in the past but the direction they went over the past 4 years is just a bad user experience. There are better music app services. Don't recommend at all."
                Topics / Keyphrases: ['Spotify App Ad Issues', 'Music Streaming Experience', 'Premium Users Non Premium Issues', 'Ads on Music Playlists', 'User Experience Feedback', 'Spotify Music Streaming Services', 'Ad Heavy Music Experience', 'Original Song Find Difficulty', 'Spotify App Closed Issue', 'Ad Free Music Streaming Service']
                Label: 1

                4. Comment: "Really glitchy lately. Just keeps stopping every 5 seconds. Then starts playing even when the app isnt open. I pay a subscription fee but am going to have to cancel. This is ridiculous now."
                Topics / Keyphrases: ['Spotify issues', 'App slow performance', 'Glitchy music streaming', 'Intrusive ads', 'Premium subscription problems', 'Music streaming experience', 'App crash and freeze issues', 'Unreliable playback', 'Music selection and recommendation', 'Subscription fee cancellation']
                Label: 1

                5. Comment: "Cannot login to my account at all. The app is repeatedly crashing. It sends the login link to my email, says it's valid for 4 hours but the link says it's expired. Nonsense."
                Topics / Keyphrases: ['Spotify login issues', 'login problems', 'account login failure', 'login error', 'authentication error', 'Spotify app issues', 'login error message', 'Spotify account problems', 'login troubleshooting', 'Spotify user experience issues']
                Label: 1

                6. Comment: "I like the app but I wanted to mostly use it for offline purposes but ever since a few days after I downloaded my download audio kept dissapearing"
                Topics / Keyphrases: ['Spotify App Issues', 'Audio Disappearance Problems', 'Offline Playback Error', 'Music Downloading Issues', 'Playlist Problems', 'Streaming Disconnection', 'App Crashing', 'Data Loss', 'Functionality Errors', 'Connection Failure']
                Label: 2

                7. Comment: "Where is my Car Mode?! You deleted it? Whyy, it was so useful, now i need to lock into phone during driving to change song.. thats stupid."
                Topics / Keyphrases: ['Spotify App Issues', 'Car Mode Problems', 'Playlist Not Working', 'Download Error', 'Application Crashing', 'Login Troubleshooting', 'Song Repeat Problems', 'Phone Connection Error', 'Music Streaming Glitches', 'Spotify App Troubleshooting']
                Label: 2

                8. Comment: "GIVE THE OPTION TO DISABLE THE VOLUME SYNC WITH SPOTIFY CONNECT FOR CHRIST SAKE!!! I have multiple Connect devices and I don't want them locked together at the same volume. THIS IS A BIG ISSUE THAT IS AN EASY FIX FOR YOU!! Listen to your (paying) users because a quick Google search shows this is a common issue. 5 stars if this petty issue is fixed."
                Topics / Keyphrases: ['Spotify App Issues', 'Volume Sync Problem', 'Music Streaming Problems', 'Premium Services Fail', 'Playback Issues', 'App Update Problems', 'Music Controls Difficulty', 'Volume Lock', 'Internet Connection Issues', 'Premium User Frustrations']
                Label: 2

                9. Comment: "It used to be so good, I had premium for years and i still would. But its not worth it anymore. Its super glitchy and buggy, even in premium. They tried to simplify it and 'have better used experience' but all that they did was make it bad. Please go back a version or two and keep it that way. I used to love the app"
                Topics / Keyphrases: ['Spotify app issues', 'premium features limitations', 'ad problems', 'buggy experience', 'music streaming frustration', 'premium subscription flaws', 'download and playlist management issues', 'unacceptable ads loudness', 'free version limitations', 'user experience problems']
                Label: 2

                10. Comment: "GREAT!app just don't like the so many apps and the constant add asking you to get Spotify premium. The other thing that annoys me is the limited skips and when I search up a song it won't let me play it it plays a playlist instead and I can't skip it all the time because I don't have anymore skips!"
                Topics / Keyphrases: ['Spotify app issues', 'Limiting skips', 'Ad-heavy music experience', 'Premium subscription limitations', 'Music discovery problems', 'User experience dissatisfaction', 'Ad annoyance', 'Song skipping frustration', 'Playlists and song selection issues', 'Music streaming limitations']
                Label: 2

                11. Comment: "The free accounts are ok. I get the ads but I really dont think it is great that you cannot choose what song you want to play with a free account. It is a good music streaming service though and I will recommend but they need to work on some of the free account features."
                Topics / Keyphrases: ['Spotify limitations', 'Spotify free account features', 'Spotify user experience', 'Music quality and ease of use', 'Spotify ads and premium membership', 'Song selection and streaming service', 'Searching and playlist management', 'Spotify feature updates and recommendations', 'Family plan and kid-friendly features', 'Free app drawbacks and limitations']
                Label: 3

                12. Comment: "I've strong network connection but sometimes this app shows there is a network problem so that lyrics can't be load. Don't know why does it happens! And after 2 or 3 Times reboot, the app shows the lyrics, you know when you're looking for lyrics and lyrics aren't here, it ruins the mood"
                Topics / Keyphrases: ['Spotify crashes', 'Music streaming problems', 'Spotify app issues', 'App freezing and bugs', 'Network connection problems', 'Spotify update issues', 'Playback problems and bugs', 'Disrupted mood and disappointment', 'App not working and glitches', 'Music playback issues']
                Label: 3

                13. Comment: "It has alot of good songs but there is the point that only premeum can skip ads and have free skips and they add songs of their own that they think you'll like but i hate them its a great app apart from that"
                Topics / Keyphrases: ['Spotify app issues', 'Offline listening limitations', 'Music streaming problems', 'Premium features', 'Ad-free experience', 'Song selection limitations', 'App functionality issues', 'Internet connection requirements', 'Music suggestion problems', 'Premium subscription limitations']
                Label: 3

                14. Comment: "The app works well and I don't mind the ads. But I think that it should have an option to play in shuffle or play in the order the songs are in; I make my playlists in the order I like them, so shuffle being the only option makes it frustrating."
                Topics / Keyphrases: ['Spotify app issues', 'Terrible user experience', 'Advertisements and skips', 'Music streaming limitations', 'Free version limitations', 'Playlist organization and customization', 'Offline listening and playback', 'Volume control and multitasking', 'Premium subscription features and limitations', 'User interface and feature requests']
                Label: 3

                15. Comment: "I've loved the app so far, the interface... organization..but I was logged out this evening and it's refusing to log me in. This is sad. Please fix it"
                Topics / Keyphrases: ['Spotify login issues', 'Premium subscription troubles', 'Google login authentication', 'Spotify app connectivity problems', 'Internet connection errors', 'Login credential problems', 'Spotify app interface issues', 'Account access difficulties', 'Spotify app storage problems', 'Offline mode difficulties']
                Label: 3

                16. Comment: "Great app, great music, love the weekly custom play lists but only negative thing is the pop aps if it not a primum user"
                Topics / Keyphrases: ['Spotify App Ad Issues', 'Music Streaming Experience', 'Premium Users Non Premium Issues', 'Ads on Music Playlists', 'User Experience Feedback', 'Spotify Music Streaming Services', 'Ad Heavy Music Experience', 'Original Song Find Difficulty', 'Spotify App Closed Issue', 'Ad Free Music Streaming Service']
                Label: 4

                17. Comment: "I love Spotify but it gets annoying when I try to add an album to a playlist, I have to go through every song and add them by hand. Other than that I find it wonderful 😊"
                Topics / Keyphrases: ['Spotify Music Streaming', 'Add Songs to Playlist', 'Offline Music Playback', 'User Experience Issues', 'Music Library Management', 'Custom Playlist Creation', 'Music Recommendation', 'Song Selection and Playback', 'Playlist Organization', 'Music Streaming Features']
                Label: 4

                18. Comment: "I may not remember the song's name so it would be good to have a function where we can just say some key words and the artist. But is amazing."
                Topics / Keyphrases: ['music streaming service improvement', 'song recognition features', 'customizable playback options', 'Spotify app issues', 'advertising and interruptions', 'offline listening capabilities', 'music library and discovery', 'app functionality and user satisfaction', 'music selection and variety', 'user experience and feedback']
                Label: 4

                19. Comment: "Love it! Great among quality, perfect sound for songs. My one problem is when it adds songs to your playlists. Other than that its perfect."
                Topics / Keyphrases: ['Spotify Music Streaming', 'Sound Quality Issues', 'Premium Features Availability', 'User Experience Problems', 'Volume Control Discomfort', 'Advertising Frequency Concerns', 'Music Recommendations Quality', 'Data Usage Impact', 'Music Streaming Service Satisfaction', 'Platform Limitations']
                Label: 4

                20. Comment: "This is a good music app overall but the only thing anoying is there's so much ads that say thank you for listening to Spotify it's so anoying it's like made.ti anoy people but overall best music app ever"
                Topics / Keyphrases: ['Spotify music app', 'best music streaming service', 'ads annoyance', 'easy use experience', 'new music discovery', 'affordable premium', 'good music quality', 'user satisfaction', 'music streaming benefits', 'ad-free experience']
                Label: 4

                21. Comment: "SPOTIFY IS MY DAMN JAM!!! I'D PICK IT OVER ANY OTHER MUSIC APP OUT THERE ESPECIALLY NOW THAT THEY HAVE ADDED THE LYRICS TO SONGS!!! My only request is to have the option to keep ur smartphone screen awake if u have the lyrics pulled up!! Love love love you Spotify Folk!!!!"
                Topics / Keyphrases: ['Spotify music app', 'music streaming platform', 'music discovery and recommendation', 'favorite songs and artists', 'playlists and user experience', 'music lyrics and feedback', 'smartphone music app', 'premium music streaming', 'music community and podcasting', 'music streaming quality and personalization']
                Label: 5

                22. Comment: "The app is great i regualry listen to lots of fantastic music and premium is very good because there are no adds! So i definetley reccomend you install it!:):):):)"
                Topics / Keyphrases: ['Spotify App', 'Great', 'Music', 'Fantastic Music', 'Premium', 'No Ads', 'Recommend', 'Install', 'Ad-free experience', 'Music recommendation']
                Label: 5

                23. Comment: "I have this app for 8 years and counting and I cannot live without it! Spotify accompanies me everywhere I go. Thank you!"
                Topics / Keyphrases: ['Music's Ever-Present Gratitude', 'Long-Term Spotify Use Habits', 'Music Knowledge Expansion', 'Pre-Internet Music Education', 'Can't Live Without Music Apps', 'Streaming Services' Evolution', 'Internet's Impact on Music Knowledge', 'The Power of Music Companion', 'Digital Music's Availability', 'Thank You Spotify for Music's Growth']
                Label: 5

                24. Comment: "so far this is the best music app I have used rarely crash's Plays offline has a multitude of features and functions and very easy to manuver around. Thanks so much guy's and gal's for all your hard work"
                Topics / Keyphrases: ['Spotify User Experience', 'Music App Personalization', 'Easy Navigation', 'High-Quality Music Stream', 'Streamlined Music Discovery', 'Offline Playback Convenience', 'App Usability and Ease of Use', 'Music App Development', 'User Feedback and Satisfaction', 'Convenient Music Access']
                Label: 5

                25. Comment: "I just Love Spotify. It has every single song/music that is, was, and will be a part of my life's music repritoire. Thanks for supplying such a great array of listening pleasures, SPOTIFY!!"
                Topics / Keyphrases: ['personalized music experience', 'Spotify music streaming', 'curated playlist', 'music discovery', 'favorite songs', 'music importance', 'music streaming app', 'weekly playlist', 'music entertainment', 'music serendipity']
                Label: 5


                Please provide the sentiment classification as a single number (1, 2, 3, 4, or 5) and nothing more for the following comment:

                Comment: "{text}"
                Topics / Keyphrases: {topics}
                Label: ?
                """
    print("\n\n")
    print(string_message)
    print("\n")
    
    chat_completion = client.chat.completions.create(
        messages = [
            {
                "role": "user",
                "content": f"""{string_message}"""
            }
    
        ],
        model="llama3-8b-8192",
        # model="mixtral-8x7b-32768",
    )
    
    armaz = chat_completion.choices[0].message.content
    print(f'{index}:\t{armaz}')
    list.append(armaz)

In [ ]:
len(list)

In [ ]:
rest = train_data[len(list):]
for index, row in rest.iterrows():
    text = row['content']
    topics = row['Merged keyphrases_reduced 2']
    string_message = f"""I need you to evaluate and assess the comment along with the set of topics/keyphrases, categorizing their sentiment into one of five levels: 1 - very negative, 2 - negative, 3 - neutral, 4 - positive, 5 - very positive. Here are examples to guide your classification:


                1. Comment: "Terrible. I won't repeat what the other reviews have said, the continuous ads are so intrusive and time-consuming with the free version. Now that they are joining the likes of other silicone valley politics and starting to censor people, I am officially done with Spotify. I was honestly considering premium to avoid the ads, but then the censorship started."
                Topics / Keyphrases: ['Spotify app issues', 'Invasive ads on free version', 'Commercial increase and bias', 'Censorship and control of information', 'Freedom of speech and fair advertising', 'Spotify's subscription and user choice', 'App controversies and linking issues', 'Silicon Valley politics and app controversies', 'Ad overload and diversity in job advertising', 'Cancel culture and free speech']
                Label: 1

                2. Comment: "ads and limited skips are annoying. you could literally download songs from free music apps and listen to it offline, unlimited skips and choose whatever songs you want. i used Spotify for a few weeks now and i think I'll go back using Music instead of this app."
                Topics / Keyphrases: ['Spotify app issues', 'Terrible user experience', 'Advertisements and skips', 'Music streaming limitations', 'Free version limitations', 'Playlist organization and customization', 'Offline listening and playback', 'Volume control and multitasking', 'Premium subscription features and limitations', 'User interface and feature requests']
                Label: 1

                3. Comment: "Poor music site with bad ethics. They were better in the past but the direction they went over the past 4 years is just a bad user experience. There are better music app services. Don't recommend at all."
                Topics / Keyphrases: ['Spotify App Ad Issues', 'Music Streaming Experience', 'Premium Users Non Premium Issues', 'Ads on Music Playlists', 'User Experience Feedback', 'Spotify Music Streaming Services', 'Ad Heavy Music Experience', 'Original Song Find Difficulty', 'Spotify App Closed Issue', 'Ad Free Music Streaming Service']
                Label: 1

                4. Comment: "Really glitchy lately. Just keeps stopping every 5 seconds. Then starts playing even when the app isnt open. I pay a subscription fee but am going to have to cancel. This is ridiculous now."
                Topics / Keyphrases: ['Spotify issues', 'App slow performance', 'Glitchy music streaming', 'Intrusive ads', 'Premium subscription problems', 'Music streaming experience', 'App crash and freeze issues', 'Unreliable playback', 'Music selection and recommendation', 'Subscription fee cancellation']
                Label: 1

                5. Comment: "Cannot login to my account at all. The app is repeatedly crashing. It sends the login link to my email, says it's valid for 4 hours but the link says it's expired. Nonsense."
                Topics / Keyphrases: ['Spotify login issues', 'login problems', 'account login failure', 'login error', 'authentication error', 'Spotify app issues', 'login error message', 'Spotify account problems', 'login troubleshooting', 'Spotify user experience issues']
                Label: 1

                6. Comment: "I like the app but I wanted to mostly use it for offline purposes but ever since a few days after I downloaded my download audio kept dissapearing"
                Topics / Keyphrases: ['Spotify App Issues', 'Audio Disappearance Problems', 'Offline Playback Error', 'Music Downloading Issues', 'Playlist Problems', 'Streaming Disconnection', 'App Crashing', 'Data Loss', 'Functionality Errors', 'Connection Failure']
                Label: 2

                7. Comment: "Where is my Car Mode?! You deleted it? Whyy, it was so useful, now i need to lock into phone during driving to change song.. thats stupid."
                Topics / Keyphrases: ['Spotify App Issues', 'Car Mode Problems', 'Playlist Not Working', 'Download Error', 'Application Crashing', 'Login Troubleshooting', 'Song Repeat Problems', 'Phone Connection Error', 'Music Streaming Glitches', 'Spotify App Troubleshooting']
                Label: 2

                8. Comment: "GIVE THE OPTION TO DISABLE THE VOLUME SYNC WITH SPOTIFY CONNECT FOR CHRIST SAKE!!! I have multiple Connect devices and I don't want them locked together at the same volume. THIS IS A BIG ISSUE THAT IS AN EASY FIX FOR YOU!! Listen to your (paying) users because a quick Google search shows this is a common issue. 5 stars if this petty issue is fixed."
                Topics / Keyphrases: ['Spotify App Issues', 'Volume Sync Problem', 'Music Streaming Problems', 'Premium Services Fail', 'Playback Issues', 'App Update Problems', 'Music Controls Difficulty', 'Volume Lock', 'Internet Connection Issues', 'Premium User Frustrations']
                Label: 2

                9. Comment: "It used to be so good, I had premium for years and i still would. But its not worth it anymore. Its super glitchy and buggy, even in premium. They tried to simplify it and 'have better used experience' but all that they did was make it bad. Please go back a version or two and keep it that way. I used to love the app"
                Topics / Keyphrases: ['Spotify app issues', 'premium features limitations', 'ad problems', 'buggy experience', 'music streaming frustration', 'premium subscription flaws', 'download and playlist management issues', 'unacceptable ads loudness', 'free version limitations', 'user experience problems']
                Label: 2

                10. Comment: "GREAT!app just don't like the so many apps and the constant add asking you to get Spotify premium. The other thing that annoys me is the limited skips and when I search up a song it won't let me play it it plays a playlist instead and I can't skip it all the time because I don't have anymore skips!"
                Topics / Keyphrases: ['Spotify app issues', 'Limiting skips', 'Ad-heavy music experience', 'Premium subscription limitations', 'Music discovery problems', 'User experience dissatisfaction', 'Ad annoyance', 'Song skipping frustration', 'Playlists and song selection issues', 'Music streaming limitations']
                Label: 2

                11. Comment: "The free accounts are ok. I get the ads but I really dont think it is great that you cannot choose what song you want to play with a free account. It is a good music streaming service though and I will recommend but they need to work on some of the free account features."
                Topics / Keyphrases: ['Spotify limitations', 'Spotify free account features', 'Spotify user experience', 'Music quality and ease of use', 'Spotify ads and premium membership', 'Song selection and streaming service', 'Searching and playlist management', 'Spotify feature updates and recommendations', 'Family plan and kid-friendly features', 'Free app drawbacks and limitations']
                Label: 3

                12. Comment: "I've strong network connection but sometimes this app shows there is a network problem so that lyrics can't be load. Don't know why does it happens! And after 2 or 3 Times reboot, the app shows the lyrics, you know when you're looking for lyrics and lyrics aren't here, it ruins the mood"
                Topics / Keyphrases: ['Spotify crashes', 'Music streaming problems', 'Spotify app issues', 'App freezing and bugs', 'Network connection problems', 'Spotify update issues', 'Playback problems and bugs', 'Disrupted mood and disappointment', 'App not working and glitches', 'Music playback issues']
                Label: 3

                13. Comment: "It has alot of good songs but there is the point that only premeum can skip ads and have free skips and they add songs of their own that they think you'll like but i hate them its a great app apart from that"
                Topics / Keyphrases: ['Spotify app issues', 'Offline listening limitations', 'Music streaming problems', 'Premium features', 'Ad-free experience', 'Song selection limitations', 'App functionality issues', 'Internet connection requirements', 'Music suggestion problems', 'Premium subscription limitations']
                Label: 3

                14. Comment: "The app works well and I don't mind the ads. But I think that it should have an option to play in shuffle or play in the order the songs are in; I make my playlists in the order I like them, so shuffle being the only option makes it frustrating."
                Topics / Keyphrases: ['Spotify app issues', 'Terrible user experience', 'Advertisements and skips', 'Music streaming limitations', 'Free version limitations', 'Playlist organization and customization', 'Offline listening and playback', 'Volume control and multitasking', 'Premium subscription features and limitations', 'User interface and feature requests']
                Label: 3

                15. Comment: "I've loved the app so far, the interface... organization..but I was logged out this evening and it's refusing to log me in. This is sad. Please fix it"
                Topics / Keyphrases: ['Spotify login issues', 'Premium subscription troubles', 'Google login authentication', 'Spotify app connectivity problems', 'Internet connection errors', 'Login credential problems', 'Spotify app interface issues', 'Account access difficulties', 'Spotify app storage problems', 'Offline mode difficulties']
                Label: 3

                16. Comment: "Great app, great music, love the weekly custom play lists but only negative thing is the pop aps if it not a primum user"
                Topics / Keyphrases: ['Spotify App Ad Issues', 'Music Streaming Experience', 'Premium Users Non Premium Issues', 'Ads on Music Playlists', 'User Experience Feedback', 'Spotify Music Streaming Services', 'Ad Heavy Music Experience', 'Original Song Find Difficulty', 'Spotify App Closed Issue', 'Ad Free Music Streaming Service']
                Label: 4

                17. Comment: "I love Spotify but it gets annoying when I try to add an album to a playlist, I have to go through every song and add them by hand. Other than that I find it wonderful 😊"
                Topics / Keyphrases: ['Spotify Music Streaming', 'Add Songs to Playlist', 'Offline Music Playback', 'User Experience Issues', 'Music Library Management', 'Custom Playlist Creation', 'Music Recommendation', 'Song Selection and Playback', 'Playlist Organization', 'Music Streaming Features']
                Label: 4

                18. Comment: "I may not remember the song's name so it would be good to have a function where we can just say some key words and the artist. But is amazing."
                Topics / Keyphrases: ['music streaming service improvement', 'song recognition features', 'customizable playback options', 'Spotify app issues', 'advertising and interruptions', 'offline listening capabilities', 'music library and discovery', 'app functionality and user satisfaction', 'music selection and variety', 'user experience and feedback']
                Label: 4

                19. Comment: "Love it! Great among quality, perfect sound for songs. My one problem is when it adds songs to your playlists. Other than that its perfect."
                Topics / Keyphrases: ['Spotify Music Streaming', 'Sound Quality Issues', 'Premium Features Availability', 'User Experience Problems', 'Volume Control Discomfort', 'Advertising Frequency Concerns', 'Music Recommendations Quality', 'Data Usage Impact', 'Music Streaming Service Satisfaction', 'Platform Limitations']
                Label: 4

                20. Comment: "This is a good music app overall but the only thing anoying is there's so much ads that say thank you for listening to Spotify it's so anoying it's like made.ti anoy people but overall best music app ever"
                Topics / Keyphrases: ['Spotify music app', 'best music streaming service', 'ads annoyance', 'easy use experience', 'new music discovery', 'affordable premium', 'good music quality', 'user satisfaction', 'music streaming benefits', 'ad-free experience']
                Label: 4

                21. Comment: "SPOTIFY IS MY DAMN JAM!!! I'D PICK IT OVER ANY OTHER MUSIC APP OUT THERE ESPECIALLY NOW THAT THEY HAVE ADDED THE LYRICS TO SONGS!!! My only request is to have the option to keep ur smartphone screen awake if u have the lyrics pulled up!! Love love love you Spotify Folk!!!!"
                Topics / Keyphrases: ['Spotify music app', 'music streaming platform', 'music discovery and recommendation', 'favorite songs and artists', 'playlists and user experience', 'music lyrics and feedback', 'smartphone music app', 'premium music streaming', 'music community and podcasting', 'music streaming quality and personalization']
                Label: 5

                22. Comment: "The app is great i regualry listen to lots of fantastic music and premium is very good because there are no adds! So i definetley reccomend you install it!:):):):)"
                Topics / Keyphrases: ['Spotify App', 'Great', 'Music', 'Fantastic Music', 'Premium', 'No Ads', 'Recommend', 'Install', 'Ad-free experience', 'Music recommendation']
                Label: 5

                23. Comment: "I have this app for 8 years and counting and I cannot live without it! Spotify accompanies me everywhere I go. Thank you!"
                Topics / Keyphrases: ['Music's Ever-Present Gratitude', 'Long-Term Spotify Use Habits', 'Music Knowledge Expansion', 'Pre-Internet Music Education', 'Can't Live Without Music Apps', 'Streaming Services' Evolution', 'Internet's Impact on Music Knowledge', 'The Power of Music Companion', 'Digital Music's Availability', 'Thank You Spotify for Music's Growth']
                Label: 5

                24. Comment: "so far this is the best music app I have used rarely crash's Plays offline has a multitude of features and functions and very easy to manuver around. Thanks so much guy's and gal's for all your hard work"
                Topics / Keyphrases: ['Spotify User Experience', 'Music App Personalization', 'Easy Navigation', 'High-Quality Music Stream', 'Streamlined Music Discovery', 'Offline Playback Convenience', 'App Usability and Ease of Use', 'Music App Development', 'User Feedback and Satisfaction', 'Convenient Music Access']
                Label: 5

                25. Comment: "I just Love Spotify. It has every single song/music that is, was, and will be a part of my life's music repritoire. Thanks for supplying such a great array of listening pleasures, SPOTIFY!!"
                Topics / Keyphrases: ['personalized music experience', 'Spotify music streaming', 'curated playlist', 'music discovery', 'favorite songs', 'music importance', 'music streaming app', 'weekly playlist', 'music entertainment', 'music serendipity']
                Label: 5


                Please provide the sentiment classification as a single number (1, 2, 3, 4, or 5) and nothing more for the following comment:

                Comment: "{text}"
                Topics / Keyphrases: {topics}
                Label: ?
                """
    print("\n\n")
    print(string_message)
    print("\n")
    
    chat_completion = client.chat.completions.create(
        messages = [
            {
                "role": "user",
                "content": f"""{string_message}"""
            }
    
        ],
        model="llama3-8b-8192",
        # model="mixtral-8x7b-32768",
    )
    
    armaz = chat_completion.choices[0].message.content
    print(f'{index}:\t{armaz}')
    list.append(armaz)

In [ ]:
len(list)

In [ ]:
# list.pop()

In [ ]:
list

In [ ]:
train_data['predictedSentiment'] = pd.Series(list)

In [ ]:
train_data

In [35]:
import re

def contains_number(s):
    return any(char.isdigit() for char in s)

def findNumbers(string):
    # Find all numbers in the string
    numbers = None
    if(contains_number(string)):
        numbers = re.findall(r'\d+', string)
    else:
        numbers = ""
    
    # Convert the numbers to integers
    
    numbers = [int(x) for x in numbers]
    
    if(len(numbers)>0):
        x = numbers[0]-1
    else:
        x = -1
    return x

In [36]:
formated = train_data['predictedSentiment'].apply(findNumbers)

In [ ]:
train_data['predictedSentiment'] = formated

In [ ]:
train_data

In [39]:
train_data.to_csv('/llamaOutput/topics/filenameSpotifyFewShotLlama-T.csv', index=False)

In [106]:
results = pd.read_csv("/llamaOutput/topics/filenameSpotifyFewShotLlama-T.csv")

In [ ]:
results

In [ ]:
results['predictedSentiment'].value_counts().sort_index()

In [109]:
filtered_df = results[results["predictedSentiment"] == -1]

In [ ]:
filtered_df

In [111]:
newResults = results[results["predictedSentiment"] != -1]

In [ ]:
newResults

In [113]:
from matplotlib.ticker import PercentFormatter
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

def cm_analysis(y_true, y_pred, filename, labels, classes, ymap=None, figsize=(17,17)):
    """
    Generate matrix plot of confusion matrix with pretty annotations.
    The plot image is saved to disk.
    args: 
      y_true:    true label of the data, with shape (nsamples,)
      y_pred:    prediction of the data, with shape (nsamples,)
      filename:  filename of figure file to save
      labels:    string array, name the order of class labels in the confusion matrix.
                 use `clf.classes_` if using scikit-learn models.
                 with shape (nclass,).
      classes:   aliases for the labels. String array to be shown in the cm plot.
      ymap:      dict: any -> string, length == nclass.
                 if not None, map the labels & ys to more understandable strings.
                 Caution: original y_true, y_pred and labels must align.
      figsize:   the size of the figure plotted.
    """
    sns.set(font_scale=2.8)

    if ymap is not None:
        y_pred = [ymap[yi] for yi in y_pred]
        y_true = [ymap[yi] for yi in y_true]
        labels = [ymap[yi] for yi in labels]
    cm = confusion_matrix(y_true, y_pred, labels=labels)
    cm_sum = np.sum(cm, axis=1, keepdims=True)
    cm_perc = cm / cm_sum.astype(float) * 100
    annot = np.empty_like(cm).astype(str)
    nrows, ncols = cm.shape
    for i in range(nrows):
        for j in range(ncols):
            c = cm[i, j]
            p = cm_perc[i, j]
            if i == j:
                s = cm_sum[i]
                annot[i, j] = '%.2f%%\n%d/%d' % (p, c, s)
            #elif c == 0:
            #    annot[i, j] = ''
            else:
                annot[i, j] = '%.2f%%\n%d' % (p, c)
    cm = confusion_matrix(y_true, y_pred, labels=labels, normalize='true')
    cm = pd.DataFrame(cm, index=labels, columns=labels)
    cm = cm * 100
    cm.index.name = 'True Label'
    cm.columns.name = 'Predicted Label'
    fig, ax = plt.subplots(figsize=figsize)
    plt.yticks(va='center')

    sns.heatmap(cm, annot=annot, fmt='', ax=ax, xticklabels=classes, cbar=True, cbar_kws={'format':PercentFormatter()}, yticklabels=classes, cmap="Blues")
    plt.savefig(f'{filename}.png',  bbox_inches='tight')
    plt.savefig(f'{filename}.svg',  bbox_inches='tight')

In [114]:
# Generate the confusion matrix
unique_labels = np.sort(newResults['scoreSA'].unique())
mapping = {0: 'Very Negative (1)', 1: 'Negative (2)', 2: 'Neutral (3)', 3: 'Positive (4)', 4: 'Very Positive (5)'}

# Applying mapping
classes = [mapping[value] for value in unique_labels]

In [ ]:
cm_analysis(newResults['scoreSA'], newResults['predictedSentiment'], 'API', unique_labels, classes)

In [ ]:
accuracy

In [ ]:
balanced_accuracy = balanced_accuracy_score(results['scoreSA'], results['predictedSentiment'])

In [ ]:
# Calculate accuracy for each class
for i in range(len(unique_labels)):
    temp = newResults[newResults["scoreSA"] == i]
    value = accuracy_score(temp['scoreSA'], temp['predictedSentiment'])
    print(f'Label:\t{i}\t-\tValue:\t{value}')

In [ ]:
balanced_accuracy

In [ ]:
precision_ALL

In [ ]:
recall_ALL

In [ ]:
f1_ALL

In [ ]:
mae_ALL

In [ ]:
precision

In [ ]:
recall

In [ ]:
f1

In [ ]:
support